# Proyecto: Joins con múltiples Data Frames

En la presente práctica veremos cómo:

* Crear Data Frames a partir archivos .csv

* Quitar el encabezado de un RDD

* Unir dos RDDs

* Ordenar valores de un Data Frame

* Renombrar columnas de un Data Frame

* Filtrar valores de un Data Frame

* Hacer JOINS (tipo SQL) con múltiples Data Frames

Algunas funciones que veremos son:

`sort( col('column_name').desc() )` Ordena un Data Frame de forma descendiente 'desc()' con respecto a los valores de la columna 'column_name'

`.distinct()` Elimina los registros repetidos en un Data Frame

`.select( col('old_column_name').alias('new_column_name') )` Selecciona y renombra las columnas de un Data Frame


In [1]:
# libreria para crear punto de conexión:
from pyspark import SparkContext

# Cargamos libreria para crear Data Frames:
from pyspark.sql import SQLContext

#from pyspark.sql import SparkSession

# Cargamos librerias para crear el schema del DataFrame
from pyspark.sql.types import StructType, StructField

# Cargamos los tipos de datos que usaremos para crear las columnas de los dataframes:
from pyspark.sql.types import  IntegerType, StringType, FloatType

#from pyspark.sql.types import Row

In [2]:
# Creamos el punto de conexió a Spark (que se ejecutará en mi máquina 'local'):
spark = SparkContext(master='local', appName='manipulando_DataFrames')

In [3]:
# Creamos el contexto para SQL:
sqlContext = SQLContext(spark)

/home/marck/.local/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
# Visualizamos el contenido de la carpeta 'Data' 
!ls ./Data/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [5]:
# Ruta en donde se encuentran los datos con los que trabajaremos:
path = './Data/'

In [6]:
# Creamos función para eliminar encabezados en RDDs:
def eliminaEncabezado(indice , interador):
    return iter( list(interador)[1:] )

In [7]:
# Número de registros a visualizar:
N=5

## Creación de un DataFrame con los datos del archivo *paises.csv*

In [8]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/paises.csv

id,equipo,sigla
1,30. Februar,AUT
2,A North American Team,MEX
3,Acipactli,MEX
4,Acturus,ARG


In [9]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
paisesSchema = StructType([
    StructField('id',IntegerType(),False),
    StructField('equipo',StringType(),False),
    StructField('sigla',StringType(),False) 
])

# Creamos el DataFrame:
paisesDF = sqlContext.read.schema(paisesSchema).option('header', 'true').csv(path+'paises.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
paisesDF.show(N)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



## Creación de un DataFrame con los datos del archivo *deporte.csv*

In [10]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/deporte.csv

deporte_id,deporte
1,Basketball
2,Judo
3,Football
4,Tug-Of-War


In [11]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
deportesSchema = StructType([
    StructField('deporte_id',IntegerType(),False),
    StructField('deporte',StringType(),False) 
])

# Creamos el DataFrame:
deportesDF = sqlContext.read.schema(deportesSchema).option('header', 'true').csv(path+'deporte.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
deportesDF.show(N)

+----------+-------------+
|deporte_id|      deporte|
+----------+-------------+
|         1|   Basketball|
|         2|         Judo|
|         3|     Football|
|         4|   Tug-Of-War|
|         5|Speed Skating|
+----------+-------------+
only showing top 5 rows



## Creación de un DataFrame con los datos del archivo *evento.csv*

In [12]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/evento.csv

evento_id,evento,deporte_id
1,Basketball Men's Basketball,1
2,Judo Men's Extra-Lightweight,2
3,Football Men's Football,3
4,Tug-Of-War Men's Tug-Of-War,4


In [13]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
eventoSchema = StructType([
    StructField('evento_id',IntegerType(),False),
    StructField('nombre',StringType(),False),
    StructField('deporte_id',IntegerType(),False) 
])

# Creamos el DataFrame:
deportesOlimpicosDF = sqlContext.read.schema(eventoSchema).option('header', 'true').csv(path+'evento.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
deportesOlimpicosDF.show(N)

+---------+--------------------+----------+
|evento_id|              nombre|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 5 rows



## Creación de un DataFrame con los datos del archivo *resultados.csv*

In [14]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/resultados.csv

resultado_id,medalla,deportista_id,juego_id,evento_id
1,NA,1,39,1
2,NA,2,49,2
3,NA,3,7,3
4,Gold,4,2,4


In [15]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
resultadoSchema = StructType([
    StructField('resultado_id',IntegerType(),False),
    StructField('medalla',StringType(),False),
    StructField('deportista_id',IntegerType(),False),
    StructField('juego_id',IntegerType(),False),
    StructField('evento_id',IntegerType(),False)
])

# Creamos el DataFrame:
resultadoDF = sqlContext.read.schema(resultadoSchema).option('header', 'true').csv(path+'resultados.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
resultadoDF.show(N)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



## Creación de un DataFrame con los datos del archivo *juegos.csv*

In [16]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/juegos.csv

,nombre_juego,annio,temporada,ciudad
1,1896 Verano,1896,Verano,Athina
2,1900 Verano,1900,Verano,Paris
3,1904 Verano,1904,Verano,St. Louis
4,1906 Verano,1906,Verano,Athina


In [17]:
# Guardamos archivo.csv en un DataFrame:

# Creamos un Schema para el DataFrame, es decir, 
# Asignamos nombre a las columnas, especificamos su tipo de dato y especificamos si el campo puede ser nulo
juegoSchema = StructType([
    StructField('juego_id',IntegerType(),False),
    StructField('nombre_juego',StringType(),False),
    StructField('anio',IntegerType(),False),
    StructField('temporada',StringType(),False),
    StructField('ciudad',StringType(),False)
])

# Creamos el DataFrame:
juegoDF = sqlContext.read.schema(juegoSchema).option('header', 'true').csv(path+'juegos.csv')

# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
juegoDF.show(N)

+--------+------------+----+---------+---------+
|juego_id|nombre_juego|anio|temporada|   ciudad|
+--------+------------+----+---------+---------+
|       1| 1896 Verano|1896|   Verano|   Athina|
|       2| 1900 Verano|1900|   Verano|    Paris|
|       3| 1904 Verano|1904|   Verano|St. Louis|
|       4| 1906 Verano|1906|   Verano|   Athina|
|       5| 1908 Verano|1908|   Verano|   London|
+--------+------------+----+---------+---------+
only showing top 5 rows



In [18]:
# Eliminamos la una columna del Data Frame:
juegoDF = juegoDF.drop('nombre_juego')

juegoDF.show(N)

+--------+----+---------+---------+
|juego_id|anio|temporada|   ciudad|
+--------+----+---------+---------+
|       1|1896|   Verano|   Athina|
|       2|1900|   Verano|    Paris|
|       3|1904|   Verano|St. Louis|
|       4|1906|   Verano|   Athina|
|       5|1908|   Verano|   London|
+--------+----+---------+---------+
only showing top 5 rows



## Creación de un DataFrame con los datos de los archivos *deportista.csv* y *deportista2.csv*

In [19]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/deportista.csv

deportista_id,nombre,genero,edad,altura,peso,equipo_id
1,A Dijiang,1,24,180,80,199
2,A Lamusi,1,23,170,60,199
3,Gunnar Nielsen Aaby,1,24,0,0,273
4,Edgar Lindenau Aabye,1,34,0,0,278


In [20]:
# Exploramos archivo.cvs para verificar si trae encabezado:

!head -n 5 Data/deportista2.csv

67787,Lee BongJu,1,27,167,56,970
67788,Lee BuTi,1,23,164,54,203
67789,Anthony N. Buddy Lee,1,34,172,62,1096
67790,Alfred A. Butch Lee Porter,1,19,186,80,825
67791,Lee ByeongGu,1,22,175,68,970


Notemos que el archivo 'deportista.csv' tiene encabezado y 'deportista2.csv' no tiene

El archivo 'deportista2.csv' es continuación de 'deportista.csv'

Para hacer un sólo Data Frame, haremos lo siguiente: 

1. Tranformamos los archivos.csv a RDDs

2. Eliminamos el encabezado del archivo 'deportista.csv'

3. Unimos los RDDs 'deportista' y 'deportista2

4. Ya que tenemos un sólo RDD lo pasamos a un Data Frame

In [21]:
# Guardamos archivos.csv en RDDs:
# .map(lambda l : l.split(',')) <-- Le asigan formato al contenido de archivos .csv

deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv').map(lambda l : l.split(','))

deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv').map(lambda l : l.split(','))

In [22]:
# Vemos los primeros N registros del RDD:
deportistaOlimpicoRDD.take(N)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278']]

In [23]:
# Eliminamos encabezado al primer RDD :
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

# Vemos los primeros N registros del RDD:
deportistaOlimpicoRDD.take(N)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [24]:
# Vemos los primeros N registros del segundo RDD:
deportistaOlimpicoRDD2.take(N)

[['67787', 'Lee BongJu', '1', '27', '167', '56', '970'],
 ['67788', 'Lee BuTi', '1', '23', '164', '54', '203'],
 ['67789', 'Anthony N. Buddy Lee', '1', '34', '172', '62', '1096'],
 ['67790', 'Alfred A. Butch Lee Porter', '1', '19', '186', '80', '825'],
 ['67791', 'Lee ByeongGu', '1', '22', '175', '68', '970']]

In [25]:
# Unimos los 2 RDDs: deportistaOlimpicoRDD y deportistaOlimpicoRDD2
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union( deportistaOlimpicoRDD2 )

In [26]:
# Creación de un Data Frame a partir de un RDD:

# Corregimos los tipos de dato en cada columna del RDD: 
deportistaOlimpicoRDD = \
    deportistaOlimpicoRDD.map(lambda x: ( int(x[0]), x[1],  int(x[2]), int(x[3]), int(x[4]), float(x[5]), int(x[6])  ) )


# A continuación creamos el schema, es decir, la estructura (columnas) del Data Frame donde pondremos los datos del RDD:
schema = StructType([
    StructField('deportista_id', IntegerType(), False ),
    StructField('nombre', StringType(), False ),
    StructField('genero', IntegerType(), False ),
    StructField('edad', IntegerType(), False ),
    StructField('altura', IntegerType(), False ),
    StructField('peso', FloatType(), False ),
    StructField('equipo_id', IntegerType(), False )
    ])

# StructField <-- instrucción para crear una columna, se debe especificar el tipo de dato que almacenará
# En caso de que el campo pueda ser nulo ponemos 'True' de lo contrario ponemos 'False'


# Creamos el DataFrame a partir del RDD:
deportistaOlimpicoDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)


# La información de RDDs se muestra con '.take()'
# La información de DataFrames se muestra con '.show()'
deportistaOlimpicoDF.show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [27]:
# Vemos los primeros N registros ordenadas de forma ascendente 
# con respecto a los valores de la columna 'edadAlJugar'

deportistaOlimpicoDF.sort('edad').show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|          133|           Franz Abb|     1|   0|     0| 0.0|      399|
|          167|Ould Lamine Abdallah|     1|   0|     0| 0.0|      362|
|           66|     Mohamed Abakkar|     1|   0|   156|48.0|     1003|
|          163|     Ismail Abdallah|     1|   0|     0| 0.0|     1095|
|          139|George Ioannis Abbot|     1|   0|     0| 0.0|     1043|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [28]:
# Nos quedamos con registros donde los valores de la columna 'edad' sean distintos de cero
deportistaOlimpicoDF = deportistaOlimpicoDF.filter( (deportistaOlimpicoDF['edad'] !=0) )

In [29]:
# Mostramos los primeros registros ordenados con respecto a los valores de la columna 'edad'
deportistaOlimpicoDF.sort('edad').show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|        71691|  Dimitrios Loundras|     1|  10|     0| 0.0|      333|
|        52070|        Etsuko Inada|     2|  11|     0| 0.0|      514|
|        40129|    Luigina Giavotti|     2|  11|     0| 0.0|      507|
|        37333|Carlos Bienvenido...|     1|  11|     0| 0.0|      982|
|        47618|Sonja Henie Toppi...|     2|  11|   155|45.0|      742|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



## Realizamos `join` entre los Data Frames: *deportistaOlimpicoDF*, *resultadoDF*, *juegoDF* y *deportesOlimpicosDF*

In [30]:
deportistaOlimpicoDF.show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [31]:
resultadoDF.show(N)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



In [32]:
juegoDF.show(N)

+--------+----+---------+---------+
|juego_id|anio|temporada|   ciudad|
+--------+----+---------+---------+
|       1|1896|   Verano|   Athina|
|       2|1900|   Verano|    Paris|
|       3|1904|   Verano|St. Louis|
|       4|1906|   Verano|   Athina|
|       5|1908|   Verano|   London|
+--------+----+---------+---------+
only showing top 5 rows



In [33]:
deportesOlimpicosDF.show(N)

+---------+--------------------+----------+
|evento_id|              nombre|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
|        5|Speed Skating Wom...|         5|
+---------+--------------------+----------+
only showing top 5 rows



In [34]:
# Hacemos un 'join' entre cuatro Data Frames:

deportistaOlimpicoDF.join(
    resultadoDF, # <-- DF de unión
    deportistaOlimpicoDF['deportista_id'] == resultadoDF['deportista_id'], # <-- campo de unión
    'left' #<-- tipo de join
    ).join(
        juegoDF,# <-- DF de unión
        juegoDF['juego_id'] == resultadoDF['juego_id'], # <-- campo de unión
        'left' #<-- tipo de join
    ).join(
        deportesOlimpicosDF,# <-- DF de unión
        deportesOlimpicosDF['evento_id'] == resultadoDF['evento_id'], # <-- campo de unión
        'left'#<-- tipo de join
    ).show(N) # <-- Muestra los primeros N registros

+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+----+---------+-----------+---------+--------------------+----------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|resultado_id|medalla|deportista_id|juego_id|evento_id|juego_id|anio|temporada|     ciudad|evento_id|              nombre|deporte_id|
+-------------+--------------------+------+----+------+----+---------+------------+-------+-------------+--------+---------+--------+----+---------+-----------+---------+--------------------+----------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|           1|     NA|            1|      39|        1|      39|1992|   Verano|  Barcelona|        1|Basketball Men's ...|         1|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|           2|     NA|            2|      49|        2|      49|2012|   Verano|     London|        2|Judo Men's Extra-.

In [35]:
# Seleccionamos y renombramos algunas columnas de la tabla anterior:

# funcones básicas de sql (permite usar la función 'col()')
from pyspark.sql.functions import *

deportistaOlimpicoDF.join(
    resultadoDF, # <-- DF de unión 
    deportistaOlimpicoDF['deportista_id'] == resultadoDF['deportista_id'], # <-- campo de unión
    'left' #<-- tipo de join
    ).join(
        juegoDF,# <-- DF de unión
        juegoDF['juego_id'] == resultadoDF['juego_id'], # <-- campo de unión
        'left' #<-- tipo de join
    ).join(
        deportesOlimpicosDF,# <-- DF de unión
        deportesOlimpicosDF['evento_id'] == resultadoDF['evento_id'], # <-- campo de unión
        'left'#<-- tipo de join
    ).select(
        deportistaOlimpicoDF['nombre'], # <-- seleccionamos columna 'nombre' de 'deportistaOlimpicoDF'
        col('edad').alias('Edad al jugar'),# <-- seleccionamos  columna 'edad' y la renombramos
        'medalla',# <-- seleccionamos columna 'medalla'
        col('anio').alias('Año de juego'),# <-- seleccionamos  columna 'anio' y la renombramos
        deportesOlimpicosDF['nombre'].alias('Nombre de disciplina') #<-- seleccionamos columna 'nombre' de 'deportesOlimpicosDF' y la renombramos
    ).show(N)# <-- Muestra los primeros N registros



    # Notemos que en la tabla anterior hay varias columnas con el mismo nombre, por esta razón debemos ser
    # más explícitos (ie. especificar a que tabla nos estamos refiriendo) al momento de hacer el 'select'

+--------------------+-------------+-------+------------+--------------------+
|              nombre|Edad al jugar|medalla|Año de juego|Nombre de disciplina|
+--------------------+-------------+-------+------------+--------------------+
|           A Dijiang|           24|     NA|        1992|Basketball Men's ...|
|            A Lamusi|           23|     NA|        2012|Judo Men's Extra-...|
| Gunnar Nielsen Aaby|           24|     NA|        1920|Football Men's Fo...|
|Edgar Lindenau Aabye|           34|   Gold|        1900|Tug-Of-War Men's ...|
|Christine Jacoba ...|           21|     NA|        1994|Speed Skating Wom...|
+--------------------+-------------+-------+------------+--------------------+
only showing top 5 rows



## Realizamos `join` entre los Data Frames: *resultadoDF*, *deportistaOlimpicoDF* y *paisesDF*

In [36]:
resultadoDF.show(N)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
|           5|     NA|            5|      36|        5|
+------------+-------+-------------+--------+---------+
only showing top 5 rows



In [37]:
deportistaOlimpicoDF.show(N)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [38]:
paisesDF.show(N)

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
+---+--------------------+-----+
only showing top 5 rows



In [39]:
# Hacemos un 'join' entre tres tablas y 
# seleccionamos unicamente los registros que tengan medalla asignada --> .filter( (resultadoDF['medalla'] != 'NA') )

resultadoDF.join(
    deportistaOlimpicoDF, # <-- DF de unión
    resultadoDF['deportista_id'] == deportistaOlimpicoDF['deportista_id'], # <-- campo de unión
    'left' #<-- tipo de join
    ).join(
        paisesDF,# <-- DF de unión
        deportistaOlimpicoDF['equipo_id'] == paisesDF['id'], # <-- campo de unión
        'left' #<-- tipo de join
    ).filter( (resultadoDF['medalla'] != 'NA') ).show(N) # <-- Muestra los primeros N registros

+------------+-------+-------------+--------+---------+-------------+--------------------+------+----+------+----+---------+---+--------------+-----+
|resultado_id|medalla|deportista_id|juego_id|evento_id|deportista_id|              nombre|genero|edad|altura|peso|equipo_id| id|        equipo|sigla|
+------------+-------+-------------+--------+---------+-------------+--------------------+------+----+------+----+---------+---+--------------+-----+
|           4|   Gold|            4|       2|        4|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|278|Denmark/Sweden|  SWE|
|          38| Bronze|           15|       7|       19|           15|Arvo Ossian Aaltonen|     1|  22|     0| 0.0|      350|350|       Finland|  FIN|
|          39| Bronze|           15|       7|       20|           15|Arvo Ossian Aaltonen|     1|  22|     0| 0.0|      350|350|       Finland|  FIN|
|          41| Bronze|           16|      50|       14|           16|Juhamatti Tapio A...|     1|  2

In [40]:
# De la tabla anterior seleccionamos únicamente las columnas:
# 'medalla',
# 'deportista_id' (del DF 'resultadoDF') renombrada como 'ID_Deportista'
# 'sigla' renombrada como 'pais' 
# 'equipo'

resultadoDF.join(
    deportistaOlimpicoDF, # <-- DF de unión
    resultadoDF['deportista_id'] == deportistaOlimpicoDF['deportista_id'], # <-- campo de unión
    'left' #<-- tipo de join
    ).join(
        paisesDF,# <-- DF de unión
        deportistaOlimpicoDF['equipo_id'] == paisesDF['id'], # <-- campo de unión
        'left' #<-- tipo de join
    ).filter( (resultadoDF['medalla'] != 'NA') )\
        .select(
            'medalla', # <-- seleccionamos columna 'medalla'
            resultadoDF['deportista_id'], # <-- seleccionamos columna 'deportista_id' del DF 'resultadoDF'
            col('sigla').alias('pais'), # <-- seleccionamos  columna 'sigla' y la renombramos
            'equipo'# <-- seleccionamos columna 'equipo'        
        ).withColumnRenamed('deportista_id','ID_Deportista')\
            .show()# <-- mostramos los primeros N registros

+-------+-------------+----+--------------+
|medalla|ID_Deportista|pais|        equipo|
+-------+-------------+----+--------------+
| Bronze|           16| FIN|       Finland|
|   Gold|           20| NOR|        Norway|
| Bronze|           20| NOR|        Norway|
| Silver|           20| NOR|        Norway|
| Bronze|           20| NOR|        Norway|
| Silver|           20| NOR|        Norway|
|   Gold|           20| NOR|        Norway|
|   Gold|           20| NOR|        Norway|
|   Gold|           20| NOR|        Norway|
| Bronze|           15| FIN|       Finland|
| Bronze|           15| FIN|       Finland|
| Bronze|           17| FIN|       Finland|
|   Gold|           17| FIN|       Finland|
|   Gold|           17| FIN|       Finland|
|   Gold|           17| FIN|       Finland|
| Bronze|           17| FIN|       Finland|
|   Gold|            4| SWE|Denmark/Sweden|
| Silver|           25| NOR|        Norway|
| Bronze|           29| NED|   Netherlands|
|   Gold|           21| NOR|    

In [41]:
# Usamos la función 'distinct()' para eliminar registros repetidos
# Usamos '.sort()' para ordenar los datos con respecto a los valores de una columna especificada

resultadoDF.join(
    deportistaOlimpicoDF, # <-- DF de unión
    resultadoDF['deportista_id'] == deportistaOlimpicoDF['deportista_id'], # <-- campo de unión
    'left' #<-- tipo de join
    ).join(
        paisesDF,# <-- DF de unión
        deportistaOlimpicoDF['equipo_id'] == paisesDF['id'], # <-- campo de unión
        'left' #<-- tipo de join
    ).filter( (resultadoDF['medalla'] != 'NA') )\
        .select(
            'medalla', # <-- seleccionamos columna 'medalla'
            resultadoDF['deportista_id'], # <-- seleccionamos columna 'deportista_id' del DF 'resultadoDF'
            col('sigla').alias('pais'), # <-- seleccionamos  columna 'sigla' y la renombramos
            'equipo'# <-- seleccionamos columna 'equipo'        
        ).withColumnRenamed('deportista_id','ID_Deportista')\
            .distinct()\
            .sort( col('pais').desc() ).show() # <-- ordenamos de forma descendente con respecto a los valores de la columna 'pais'

+-------+-------------+----+----------+
|medalla|ID_Deportista|pais|    equipo|
+-------+-------------+----+----------+
|   Gold|        94464| ZIM|  Zimbabwe|
|   Gold|        97036| ZIM|  Zimbabwe|
|   Gold|        25651| ZIM|  Zimbabwe|
|   Gold|        23570| ZIM|  Zimbabwe|
|   Gold|       127168| ZIM|  Zimbabwe|
|   Gold|        20673| ZIM|  Zimbabwe|
|   Gold|        19962| ZIM|  Zimbabwe|
|   Gold|       115129| ZIM|  Zimbabwe|
| Silver|        23549| ZIM|  Zimbabwe|
|   Gold|        23549| ZIM|  Zimbabwe|
| Bronze|        23549| ZIM|  Zimbabwe|
|   Gold|        42417| ZIM|  Zimbabwe|
|   Gold|       128977| ZIM|  Zimbabwe|
|   Gold|       101386| ZIM|  Zimbabwe|
|   Gold|        32681| ZIM|  Zimbabwe|
|   Gold|        50854| ZIM|  Zimbabwe|
|   Gold|        39559| ZIM|  Zimbabwe|
|   Gold|        77735| ZIM|  Zimbabwe|
| Silver|        76386| ZAM|    Zambia|
| Silver|        28794| YUG|Yugoslavia|
+-------+-------------+----+----------+
only showing top 20 rows



In [42]:
# 
resultadoDF.join(
    deportistaOlimpicoDF, # <-- DF de unión
    resultadoDF['deportista_id'] == deportistaOlimpicoDF['deportista_id'], # <-- campo de unión
    'left' #<-- tipo de join
    ).join(
        paisesDF,# <-- DF de unión
        deportistaOlimpicoDF['equipo_id'] == paisesDF['id'], # <-- campo de unión
        'left' #<-- tipo de join
    ).filter( (resultadoDF['medalla'] != 'NA') )\
        .select(
            'medalla', # <-- seleccionamos columna 'medalla'
            'equipo',# <-- seleccionamos columna 'equipo'
            'sigla',# <-- seleccionamos columna 'sigla'   
        ).sort( col('sigla').desc() ).show() # <-- ordenamos de forma descendente con respecto a los valores de la columna 'sigla'

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [43]:
# Cerramos sesión para liberar memoria:
spark.stop()